<a href="https://colab.research.google.com/gist/yiftachn/49329d278b380a86cfe2a92a4fe2dc76/hw3-sna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [0]:
import networkx as nx
import pandas as pd
import numpy as np
import bokeh as bk
import glob
from tqdm import tqdm
from networkx.algorithms import approximation as apx
path = "/content/gdrive/My Drive/Studies/SNA/hw3 repo/hw3-sna/data/"
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.cluster import KMeans, DBSCAN, MeanShift
from collections import Counter

# load networks

In [0]:
networks = []
for file in tqdm(glob.glob(path+"*.txt")):
  df = pd.read_csv(file,sep=" ",header = None)
  g = nx.from_pandas_edgelist(df,0,1)
  g.name = file[len(path):]
  networks.append(g)

# Calculate labes and divide train and test

In [0]:
network_labels = {}
for network in tqdm(networks):
  name = network.name
  sep = name.find('-')
  if sep == -1 :
    network_labels[name] = name[0:2]
  else:
    network_labels[name] = name[:sep]

In [0]:
type({(x,1) for x in [1,2,3]})

# calculate features

In [0]:
network_features = pd.DataFrame(dict((x.name[6:],0) for x in networks),columns=['name','average_degree']).set_index('name')


## calculate average degree by std



In [0]:
for network in networks:
  degrees = []
  for d in nx.degree(network):
    degrees.append(d[1])
  degrees = np.array(degrees)
  std_by_mean = np.std(degrees)/np.mean(degrees)
  network_features.loc[network.name,'degree_mean'] = np.mean(degrees)
  network_features.loc[network.name,'degree_std_by_mean'] = std_by_mean

## calculate connected components number


In [0]:
for network in tqdm(networks):
  network_features.loc[network.name,'connected_components'] = nx.number_connected_components(network)

## calculate clustering coefficient

In [0]:
for network in tqdm(networks):
  network_features.loc[network.name,'avg_clustering_coefficient'] = apx.clustering_coefficient.average_clustering(network)

## calculate vertex cover by size


In [0]:
for network in networks:
    network_features.loc[network.name,'vertex_cover_by_size'] = len(apx.min_weighted_vertex_cover(network))/ nx.number_of_nodes(network)


In [0]:
network_features = network_features.drop(columns = 'average_degree')

# Standratize features

In [0]:
def standart(column):
  return ((column-column.mean())/column.std())

In [0]:
def normal(column):
  return ((column-column.mean())/column.max()-column.min())

In [0]:
network_features = network_features.apply(normal,axis = 0)

# train model

In [0]:
models = [KMeans(n_clusters=12),MeanShift(),DBSCAN()]
predictions = []
for model in models:
    predictions.append(model.fit_predict(network_features))

In [0]:
predtuples = []
for prediction in predictions:
    predtuples.append([(prediction[i],network_labels[network_features.iloc[i].name]) for i in range(len(network_features))])

In [0]:
def purity(predictions):
    predictions.sort()
    clusters = set(x for x,y in predictions)
    sum = 0
    for cluster in clusters:
        counter = Counter(y for x,y in predictions if x==cluster)
        sum += max(counter.values())
    return sum /len(predictions)

In [0]:
purity(predtuples[0])

In [0]:
predtuples[0]

In [0]:
for i in range (0,13):
    c = Counter(y for x,y in predtuples if x==i)
    print(c)

# predict and evaluate